# vcf_heterozygosity.ipynb

Script to calculate heterozygosity from output of Svardal Lab VCF pipeline.

## Input and variables

In [2]:
# Import packages
import numpy as np
import pandas as pd
import pysam

In [9]:
# Define variables
path = "/scratch/antwerpen/grp/asvardal/projects/felids/data/AlignmentVariantCalling/data/Fca126/Felidhet/"
#path = "/scratch/antwerpen/grp/asvardal/projects/felids/data/AlignmentVariantCalling/data/Pti1/Felidhet/"
vcf_fn = "Felidhet.no_if1.sf_stringent1.pass.snps.biallelic.all_chrom.vcf.gz"
vcf_in = pysam.VariantFile(path + vcf_fn)

samples = [s for s in vcf_in.header.samples]
autosomes = ["CM0314" + str(i) + ".1" for i in range(12,30)] #Fca126
#autosomes = ["CM0314" + str(i) + ".1" for i in range(31,49)] #Pti1

bed_fn = ["Felidhet.no_if1.sf_stringent1." + chr + ".accessible.bed.gz" for chr in autosomes]

output_fn = "Felidhet.no_if1.sf_stringent1.pass.snps.biallelic.autosomal_heterozygosity.tsv"

## Size of the accessible genome

In [10]:
%%capture

chr_acc_size_list = []
acc_size = 0
for bed in bed_fn:
    %env BED={path+bed}
    chr_acc_size = !zcat $BED | awk 'BEGIN{tot=0} {tot=tot+$3-$2} END{print tot}'
    chr_acc_size_list.append(int(chr_acc_size[0]))
    acc_size = sum(chr_acc_size_list)
    
tot_size = 0
for bed in bed_fn:
    %env BED={path+bed}
    chr_tot_size = !zcat $BED | tail -n 1 | awk '{print $3}'
    tot_size += int(chr_tot_size[0])

In [11]:
print("Size of the accessible part of the autosomal genome: " + f'{acc_size:,}' + " bp")
print("Size of all autosomes combined:                      " + f'{tot_size:,}' + " bp")
print("Accessible fraction of the autosomal genome:         " + '{:.0%}'.format(acc_size/tot_size))

Size of the accessible part of the autosomal genome: 1,778,819,369 bp
Size of all autosomes combined:                      2,287,901,109 bp
Accessible fraction of the autosomal genome:         78%


## Calculate heterozygosity

In [12]:
# Initiate empty lists to hold libraries
hets_list = []
homs_list = []
missing_list = []

# For each autosome, create libraries with the number of het, hom and missing sites per sample 
for chr in autosomes:
    for k, rec in enumerate(vcf_in.fetch(chr)):
        if not k:
            print("Counting in " + rec.contig)
            hets = {s:0 for s in samples}
            homs = {s:0 for s in samples}
            missing = {s:0 for s in samples}
        for name, vrec in rec.samples.iteritems():
            a1, a2 = vrec.alleles
            if None in [a1, a2]:
                missing[name] += 1
            elif a1==a2:
                homs[name] += 1
            elif a1!=a2:
                hets[name] += 1
            else:
                print("Alleles:",a1, a2)
                raise Exception("Whats going on here???")

    # Add autosome-level libraries to lists
    hets_list.append(hets)
    homs_list.append(homs)
    missing_list.append(missing)

Counting in CM031431.1
Counting in CM031432.1
Counting in CM031433.1
Counting in CM031434.1
Counting in CM031435.1
Counting in CM031436.1
Counting in CM031437.1
Counting in CM031438.1
Counting in CM031439.1
Counting in CM031440.1
Counting in CM031441.1
Counting in CM031442.1
Counting in CM031443.1
Counting in CM031444.1
Counting in CM031445.1
Counting in CM031446.1
Counting in CM031447.1
Counting in CM031448.1


In [46]:
# Initiate empty lists to hold target statistics
weighted_av = []
weighted_std = []
minimum = []
maximum = []

# For each sample and autosome, transform absolute counts to relative heterozygosity
for s in samples:
    estimates = []

    for i in range(len(autosomes)):
        estimates.append((hets_list[i][s] + missing_list[i][s]
              * (hets_list[i][s] / (hets_list[i][s] + homs_list[i][s])))
              / chr_acc_size_list[i])

    # Calculate weighted average, weighted standard deviation, min and max from autosomal estimates
    weighted_av.append(np.average(estimates, weights=chr_acc_size_list))
    weighted_std.append(np.sqrt(np.cov(estimates, aweights=chr_acc_size_list)))
    minimum.append(min(estimates))
    maximum.append(max(estimates))

# Store lists as dataframe
df = pd.DataFrame(list(zip(samples, weighted_av, weighted_std, minimum, maximum)), 
                  columns=['unique_id', 'heterozygosity', 'std', 'min', 'max'])

## Write to file

In [61]:
# Write dataframe to file
with open(path + output_fn, 'w') as file:
    df.to_csv(file, sep='\t', index = False)